In [ ]:
from tkinter.ttk import Style
from vtkmodules.vtkCommonCore import VTK_UNSIGNED_CHAR, vtkPoints
from vtkmodules.vtkCommonDataModel import (vtkImageData, vtkCellArray, vtkPolyData, 
                                          vtkPolyLine, vtkOctreePointLocator, vtkPlane)
from vtkmodules.vtkFiltersGeometry import vtkImageDataGeometryFilter
from vtkmodules.vtkIOImage import vtkPNGWriter, vtkPNGReader, vtkJPEGReader
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkFiltersSources import vtkCubeSource, vtkCylinderSource, vtkSphereSource
from vtkmodules.vtkFiltersModeling import vtkOutlineFilter
from vtkmodules.vtkFiltersGeneral import vtkClipDataSet
from vtkmodules.vtkFiltersTexture import vtkTextureMapToPlane, vtkTextureMapToSphere
from vtkmodules.vtkFiltersCore import vtkImplicitPolyDataDistance
import vtkmodules.vtkRenderingOpenGL2
import vtkmodules.vtkInteractionStyle
from vtkmodules.vtkFiltersCore import vtkGlyph3D, vtkClipPolyData, vtkFeatureEdges, vtkStripper
from vtk import vtkInteractorStyleTrackballCamera, vtkObject
from vtkmodules.vtkRenderingCore import (vtkActor, vtkPolyDataMapper, vtkRenderWindow, vtkDataSetMapper,
                                        vtkRenderWindowInteractor, vtkRenderer, vtkTexture, vtkProperty)

In [37]:
def cube_mesh(size, cutObject, discretization):
    implicitPolyDataDistance = vtkImplicitPolyDataDistance()
    implicitPolyDataDistance.SetInput(cutObject.GetOutputPort())
    
    xCoords = vtkFloatArray()
    for x, i in enumerate(np.linspace(-size/2, size/2, discretization)):
        xCoords.InsertNextValue(i)

    yCoords = vtkFloatArray()
    for y, i in enumerate(np.linspace(-size/2, size/2, discretization)):
        yCoords.InsertNextValue(i)

    zCoords = vtkFloatArray()
    for z, i in enumerate(np.linspace(-size/2, size/2, discretization)):
        zCoords.InsertNextValue(i)

    # The coordinates are assigned to the rectilinear grid. Make sure that
    # the number of values in each of the XCoordinates, YCoordinates,
    # and ZCoordinates is equal to what is defined in SetDimensions().
    rgrid = vtkRectilinearGrid()
    rgrid.SetDimensions(xCoords.GetNumberOfTuples(),
                        yCoords.GetNumberOfTuples(),
                        zCoords.GetNumberOfTuples())
    rgrid.SetXCoordinates(xCoords)
    rgrid.SetYCoordinates(yCoords)
    rgrid.SetZCoordinates(zCoords)

    # Create an array to hold distance information
    signedDistances = vtkFloatArray()
    signedDistances.SetNumberOfComponents(1)
    signedDistances.SetName('SignedDistances')

    # Evaluate the signed distance function at all of the grid points
    for pointId in range(0, rgrid.GetNumberOfPoints()):
        p = rgrid.GetPoint(pointId)
        signedDistance = implicitPolyDataDistance.EvaluateFunction(p)
        signedDistances.InsertNextValue(signedDistance)

    # Add the SignedDistances to the grid
    rgrid.GetPointData().SetScalars(signedDistances)
    return rgrid

In [5]:
ren = vtkRenderer()

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)
style = vtkInteractorStyleTrackballCamera()
iren = vtkRenderWindowInteractor()
iren.SetInteractorStyle(style)
iren.SetRenderWindow(renWin)


colors = vtkNamedColors()
# Set the background color.
bkg = map(lambda x: x / 255.0, [26, 51, 102, 255])
colors.SetColor("ivory_black", *bkg)

configs = [ {"image_file": "LogoMines.jpg", "angle": 0},
            {"image_file": "LogoMines.jpg", "angle": 90},
            {"image_file": "LogoMines.jpg", "angle": 180},
            {"image_file": "LogoMines.jpg", "angle": 270},
]

for config in configs:
    reader = vtkJPEGReader()
    reader.SetFileName(config["image_file"])
    #reader = vtkPNGReader()
    #reader.SetFileName("teste.png")
    texture = vtkTexture()
    texture.SetInputConnection(reader.GetOutputPort())

    plane = vtkPlane()
    plane.SetOrigin(0.6, 0, 0)
    plane.SetNormal(1, 0, 0)

    #cube = vtkSphereSource()
    cube = vtkCubeSource()
    cube.SetXLength(1)
    cube.SetYLength(1)
    cube.SetZLength(1)
    cube.SetCenter((0.2, 0, 0))
    # cube.SetThetaResolution(12)
    # cube.SetPhiResolution(12)
    #cube = cube_mesh(1, plane, 10)


    map_to_plane = vtkTextureMapToPlane()
    # map_to_plane = vtkTextureMapToSphere()
    map_to_plane.SetInputConnection(cube.GetOutputPort())
    # map_to_plane.PreventSeamOn()

    #clipper = vtkClipPolyData()
    clipper = vtkClipDataSet()
    clipper.SetInputConnection(cube.GetOutputPort())

    # clipper.SetInputData()
    # clipper.InsideOutOn()
    clipper.GenerateClippedOutputOn()

    clipper.SetClipFunction(plane)
    clipper.SetValue(0)
    clipper.Update()

    #cutCubeData = clipper.GetOutput()


    #image_mapper = vtkPolyDataMapper()
    image_mapper = vtkDataSetMapper()
    image_mapper.SetInputConnection(clipper.GetOutputPort())
    #image_mapper.ScalarVisibilityOff()
    #image_mapper.SetInputConnection(map_to_plane.GetOutputPort())

    planeActor = vtkActor()
    planeActor.SetMapper(image_mapper)
    planeActor.SetTexture(texture)
    planeActor.DragableOn()
    planeActor.SetDragable(1)

    planeActor.GetProperty().SetColor(colors.GetColor3d("Navy"))
    # cylinderActor.RotateX(30.0)
    # cylinderActor.RotateY(-45.0)

    boundaryEdges = vtkFeatureEdges()
    boundaryEdges.SetInputConnection(clipper.GetOutputPort())
    boundaryEdges.BoundaryEdgesOn()
    boundaryEdges.FeatureEdgesOff()
    boundaryEdges.NonManifoldEdgesOff()
    boundaryEdges.ManifoldEdgesOff()

    boundaryStrips = vtkStripper()
    boundaryStrips.SetInputConnection(boundaryEdges.GetOutputPort())
    boundaryStrips.Update()

    # Change the polylines into polygons
    boundaryPoly = vtkPolyData()
    boundaryPoly.SetPoints(boundaryStrips.GetOutput().GetPoints())
    boundaryPoly.SetPolys(boundaryStrips.GetOutput().GetLines())

    boundaryMapper = vtkPolyDataMapper()
    boundaryMapper.SetInputData(boundaryPoly)

    boundaryActor = vtkActor()
    boundaryActor.SetMapper(boundaryMapper)
    boundaryActor.GetProperty().SetColor(colors.GetColor3d("Navy"))

    planeActor.RotateY(config["angle"])
    boundaryActor.RotateY(config["angle"])
    ren.AddActor(planeActor)
    ren.AddActor(boundaryActor)

mainCube =vtkCubeSource()
mainCube.SetXLength(1.2)
mainCube.SetYLength(1.2)
mainCube.SetZLength(1.2)
mainCube.SetCenter((0, 0, 0))

mainCubeMapper = vtkDataSetMapper()
mainCubeMapper.SetInputConnection(mainCube.GetOutputPort())
mainCubeActor = vtkActor()
mainCubeActor.SetMapper(mainCubeMapper)
mainCubeActor.GetProperty().SetColor(colors.GetColor3d("Navy"))

cubeRef = vtkCubeSource()
cubeRef.SetXLength(2)
cubeRef.SetYLength(2)
cubeRef.SetZLength(2)
cubeRef.SetCenter((0, 0, 0))

outline = vtkOutlineFilter()
outline.SetInputConnection(cubeRef.GetOutputPort())

outlineMapper = vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())

outlineActor = vtkActor()
outlineActor.SetMapper(outlineMapper)

# cubemap = vtkTexture()
# cubemap.CubeMapOn()

ren.AddActor(outlineActor)
ren.AddActor(mainCubeActor)

#ren.SetEnvironmentcylinderMap()
ren.SetBackground(colors.GetColor3d("BkgColor"))
renWin.SetSize(500, 500)
renWin.SetWindowName('Vtk Project')
ren.AddActor(planeActor)


C:\Users\eduardo.vital_brasil\AppData\Local\Temp\ipykernel_25984\1069472012.py:14: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  colors.SetColor("ivory_black", *bkg)


In [42]:
ren = vtkRenderer()

renWin = vtkRenderWindow()
renWin.AddRenderer(ren)
style = vtkInteractorStyleTrackballCamera()
iren = vtkRenderWindowInteractor()
iren.SetInteractorStyle(style)
iren.SetRenderWindow(renWin)

cube = vtkCubeSource()
cube.SetXLength(1)
cube.SetYLength(1)
cube.SetZLength(1)
cube.SetCenter((0.2, 0, 0))

map_to_plane = vtkTextureMapToPlane()
map_to_plane.SetInputConnection(cube.GetOutputPort())

plane = vtkPlane()
plane.SetOrigin(0.6, 0, 0)
plane.SetNormal(1, 0, 0)

clipper = vtkClipDataSet()
clipper.SetInputConnection(cube.GetOutputPort())
clipper.SetClipFunction(plane)
clipper.SetValue(0)
clipper.Update()

image_mapper = vtkDataSetMapper()
image_mapper.SetInputConnection(clipper.GetOutputPort())

planeActor = vtkActor()
planeActor.SetMapper(image_mapper)
planeActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
planeActor.GetProperty().SetEdgeColor(colors.GetColor3d("Red")) 
renWin.SetSize(500, 500)
renWin.SetWindowName('Vtk Project')
ren.AddActor(planeActor)
ren.SetBackground(colors.GetColor3d("Blue"))



In [ ]:
# This allows the interactor to initalize itself. It has to be
# called before an event loop.
iren.Initialize()

# We'll zoom in a little by accessing the camera and invoking a "Zoom"
# method on it.
ren.ResetCamera()
ren.GetActiveCamera().Zoom(0.7)
renWin.Render()

# Start the event loop.
iren.Start()

In [40]:
# colors.GetColorNames()

In [39]:
import numpy as np
# noinspection PyUnresolvedReferences
import vtkmodules.vtkInteractionStyle
# noinspection PyUnresolvedReferences
import vtkmodules.vtkRenderingOpenGL2
from vtkmodules.vtkCommonColor import vtkNamedColors
from vtkmodules.vtkCommonCore import vtkFloatArray
from vtkmodules.vtkCommonDataModel import (
    vtkCellTypes,
    vtkRectilinearGrid
)
from vtkmodules.vtkFiltersCore import vtkImplicitPolyDataDistance
from vtkmodules.vtkFiltersGeneral import vtkClipDataSet
from vtkmodules.vtkFiltersGeometry import vtkRectilinearGridGeometryFilter
from vtkmodules.vtkFiltersSources import vtkConeSource
from vtkmodules.vtkRenderingCore import (
    vtkActor,
    vtkDataSetMapper,
    vtkPolyDataMapper,
    vtkRenderWindow,
    vtkRenderWindowInteractor,
    vtkRenderer
)


def main():
    colors = vtkNamedColors()

    # Create polydata to slice the grid with. In this case, use a cone. This
    # could
    # be any polydata including a stl file.
    cone = vtkConeSource()
    cone.SetResolution(50)
    cone.SetDirection(0, 0, -1)
    cone.SetHeight(3.0)
    cone.CappingOn()
    cone.Update()

    # Implicit function that will be used to slice the mesh
    implicitPolyDataDistance = vtkImplicitPolyDataDistance()
    implicitPolyDataDistance.SetInput(cone.GetOutput())

    # create a grid
    dimension = 51
    xCoords = vtkFloatArray()
    for x, i in enumerate(np.linspace(-1.0, 1.0, dimension)):
        xCoords.InsertNextValue(i)

    yCoords = vtkFloatArray()
    for y, i in enumerate(np.linspace(-1.0, 1.0, dimension)):
        yCoords.InsertNextValue(i)

    zCoords = vtkFloatArray()
    for z, i in enumerate(np.linspace(-1.0, 1.0, dimension)):
        zCoords.InsertNextValue(i)

    # The coordinates are assigned to the rectilinear grid. Make sure that
    # the number of values in each of the XCoordinates, YCoordinates,
    # and ZCoordinates is equal to what is defined in SetDimensions().
    rgrid = vtkRectilinearGrid()
    rgrid.SetDimensions(xCoords.GetNumberOfTuples(),
                        yCoords.GetNumberOfTuples(),
                        zCoords.GetNumberOfTuples())
    rgrid.SetXCoordinates(xCoords)
    rgrid.SetYCoordinates(yCoords)
    rgrid.SetZCoordinates(zCoords)

    # Create an array to hold distance information
    signedDistances = vtkFloatArray()
    signedDistances.SetNumberOfComponents(1)
    signedDistances.SetName('SignedDistances')

    # Evaluate the signed distance function at all of the grid points
    for pointId in range(0, rgrid.GetNumberOfPoints()):
        p = rgrid.GetPoint(pointId)
        signedDistance = implicitPolyDataDistance.EvaluateFunction(p)
        signedDistances.InsertNextValue(signedDistance)

    # Add the SignedDistances to the grid
    rgrid.GetPointData().SetScalars(signedDistances)

    # Use vtkClipDataSet to slice the grid with the polydata
    clipper = vtkClipDataSet()
    clipper.SetInputData(rgrid)
    clipper.InsideOutOn()
    clipper.SetValue(0.0)
    clipper.GenerateClippedOutputOn()
    clipper.Update()

    # --- mappers, actors, render, etc. ---
    # mapper and actor to view the cone
    coneMapper = vtkPolyDataMapper()
    coneMapper.SetInputConnection(cone.GetOutputPort())
    coneActor = vtkActor()
    coneActor.SetMapper(coneMapper)

    # geometry filter to view the background grid
    geometryFilter = vtkRectilinearGridGeometryFilter()
    geometryFilter.SetInputData(rgrid)
    geometryFilter.SetExtent(0, dimension, 0, dimension, int(dimension / 2), int(dimension / 2))
    geometryFilter.Update()

    rgridMapper = vtkPolyDataMapper()
    rgridMapper.SetInputConnection(geometryFilter.GetOutputPort())
    rgridMapper.SetScalarRange(
        rgrid.GetPointData().GetArray('SignedDistances').GetRange())

    wireActor = vtkActor()
    wireActor.SetMapper(rgridMapper)
    wireActor.GetProperty().SetRepresentationToWireframe()

    # mapper and actor to view the clipped mesh
    clipperMapper = vtkDataSetMapper()
    clipperMapper.SetInputConnection(clipper.GetOutputPort())
    clipperMapper.ScalarVisibilityOff()

    clipperOutsideMapper = vtkDataSetMapper()
    clipperOutsideMapper.SetInputConnection(clipper.GetOutputPort(1))
    clipperOutsideMapper.ScalarVisibilityOff()

    clipperActor = vtkActor()
    clipperActor.SetMapper(clipperMapper)
    clipperActor.GetProperty().SetColor(colors.GetColor3d('Banana'))

    clipperOutsideActor = vtkActor()
    clipperOutsideActor.SetMapper(clipperOutsideMapper)
    clipperOutsideActor.GetProperty().SetColor(
        colors.GetColor3d('Banana'))

    # A renderer and render window
    # Create a renderer, render window, and interactor
    leftViewport = [0.0, 0.0, 0.5, 1.0]
    leftRenderer = vtkRenderer()
    leftRenderer.SetViewport(leftViewport)
    leftRenderer.SetBackground(colors.GetColor3d('SteelBlue'))

    rightViewport = [0.5, 0.0, 1.0, 1.0]
    rightRenderer = vtkRenderer()
    rightRenderer.SetViewport(rightViewport)
    rightRenderer.SetBackground(colors.GetColor3d('CadetBlue'))

    # add the actors
    leftRenderer.AddActor(wireActor)
    leftRenderer.AddActor(clipperActor)
    rightRenderer.AddActor(clipperOutsideActor)

    renwin = vtkRenderWindow()
    renwin.SetSize(640, 480)
    renwin.AddRenderer(leftRenderer)
    renwin.AddRenderer(rightRenderer)
    renwin.SetWindowName('ClipDataSetWithPolyData')

    # An interactor
    interactor = vtkRenderWindowInteractor()
    interactor.SetRenderWindow(renwin)

    # Share the camera

    leftRenderer.GetActiveCamera().SetPosition(0, -1, 0)
    leftRenderer.GetActiveCamera().SetFocalPoint(0, 0, 0)
    leftRenderer.GetActiveCamera().SetViewUp(0, 0, 1)
    leftRenderer.GetActiveCamera().Azimuth(30)
    leftRenderer.GetActiveCamera().Elevation(30)
    leftRenderer.ResetCamera()
    rightRenderer.SetActiveCamera(leftRenderer.GetActiveCamera())

    renwin.Render()
    interactor.Start()

    # Generate a report
    ct = vtkCellTypes()

    numberOfCells = clipper.GetOutput().GetNumberOfCells()
    print('------------------------')
    print('The clipped dataset(inside) contains a\n', clipper.GetOutput().GetClassName(), 'that has', numberOfCells,
          'cells')
    cellMap = dict()
    for i in range(0, numberOfCells):
        cellMap[clipper.GetOutput().GetCellType(i)] = cellMap.get(clipper.GetOutput().GetCellType(i), 0) + 1

    for k, v in cellMap.items():
        print('\tCell type ', ct.GetClassNameFromTypeId(k), 'occurs', v, 'times.')

    numberOfCells = clipper.GetClippedOutput().GetNumberOfCells()
    print('------------------------')
    print('The clipped dataset(outside) contains a\n', clipper.GetClippedOutput().GetClassName(), 'that has',
          numberOfCells, 'cells')
    outsideCellMap = dict()
    for i in range(0, numberOfCells):
        outsideCellMap[clipper.GetClippedOutput().GetCellType(i)] = outsideCellMap.get(
            clipper.GetClippedOutput().GetCellType(i), 0) + 1

    for k, v in outsideCellMap.items():
        print('\tCell type ', ct.GetClassNameFromTypeId(k), 'occurs', v, 'times.')


if __name__ == '__main__':
    main()

------------------------
The clipped dataset(inside) contains a
 vtkUnstructuredGrid that has 49514 cells
	Cell type  vtkTetra occurs 41034 times.
	Cell type  vtkWedge occurs 8480 times.
------------------------
The clipped dataset(outside) contains a
 vtkUnstructuredGrid that has 714434 cells
	Cell type  vtkTetra occurs 705090 times.
	Cell type  vtkWedge occurs 9344 times.
